In [1]:
import os
from torch.optim import AdamW
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder

C:\Users\Gumpun\AppData\Local\Temp\ipykernel_22756\196635466.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def file2list(fname):
    with open(fname, encoding='utf-8') as fp:  # Specify the encoding
        lines = fp.readlines()
        return [line.strip() for line in lines]

In [3]:
pos_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Positive.csv")
neg_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Negative.csv")
neu_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Neutrally.csv")

In [4]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
289,*******765,เสื้อตัวเล็กและผ้าบางมากๆๆๆๆ,Neutrally
819,0***8,บางไปนิดค่ะ,Neutrally
758,ภากมล แ.,สั่งสีขาวจีได้สีส้มแป๊ด ปวดหัว ร้านค้าตอบกลับด...,Negative
696,จันทนา ใ.,สินค้าเล็กมากๆๆๆๆๆๆๆใส่ไม่ได้,Negative
986,*******644,เนื้อผ้าไม่โอเคไม่ผ่าน โหลมาก,Negative
...,...,...,...
24,0***5,ส่งสินค้าเร๋วมากๆๆๆ รองเท้านุ่ม ใส่ดีสมราคาค่ะ,Positive
554,Thanakorn N.,เนื้องานด้านนอกดูโอเค แต่ข้างในไม่เนียน กระเป๋...,Neutrally
528,ร***.,กระทะใบใหญ่ดี มีรอยบุบนิดหน่อย รอยเหมือนๆรูปใน...,Neutrally
32,สุชาติ จ.,การเคลื่อนไหวที่เงียบสงบและราบรื่น ดีไซน์ที่ดู...,Positive


In [5]:
category_counts = data['Sentiment'].value_counts()
category_counts

Sentiment
Positive     1499
Negative     1496
Neutrally    1496
Name: count, dtype: int64

In [6]:
!pip install pythainlp

In [7]:
from pythainlp.tokenize import word_tokenize

In [8]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)

,User,Message,Sentiment
1122,น***.,ได้ ของ ไม่ ครบ สั่ง ได้ แค่ ตัว ไม่ ไ...,Negative
880,Natnaree2544,สั่ง ไป ตัว แต่ ได้รับ มา แค่ ตัว และ ไม่ ...,Negative
883,สมศักดิ์,ของดี สั่ง รอบ แล้ว,Positive
806,ร***.,ใส่ ได้ พอดี คะ เนื้อผา ดีมาก คะ,Neutrally
314,0***9,ไม่ ใส่ เลย คะ สินค้า เหมาะส กับ ราค,Negative
1313,มุทิตา ห.,สินค้า ไม่ ตรง ปก ผ้า บาง มาก เย็บ โหล ไม่ ดี ...,Negative
1331,Pusanaporm,พอใช้ได ตาม ราค ค่ะ,Neutrally
517,Tnat T.,สี ไม่ เหมือนกับ รอย ก่อน แถม บาง กว่า,Neutrally
839,Piyathida,จัดส่ง เร็ว พอสมควร ผ้า เบา สบาย ใน การเคลื่...,Positive
237,ภคินี โ.,ผ้า บาง มาก,Negative


In [9]:
Message = data['Message'].values
Sentiment = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)
print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")

for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [10]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
train_sentences, test_sentences, train_labels, test_labels = train_test_split(Message, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [12]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device).long()  # Convert to Long
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [13]:
#ฟังก์ชันนี้ใช้สำหรับประเมินผลลัพธ์ของโมเดลที่ถูกฝึกสอน
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions, zero_division=1)

In [27]:
 bert_model_name = 'bert-base-uncased'
 num_classes = 3
 max_length = 128
 batch_size = 16
 num_epochs = 10
 learning_rate = 2e-5

In [29]:
#คลาสที่ใช้สำหรับสร้าง Dataset สำหรับงานการจำแนกประเภทข้อความ
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [31]:
#โค้ดที่ให้มากำลังดึง tokenizer และสร้าง Dataset และ DataLoader สำหรับการฝึกและการทดสอบโมเดล
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_sentences, train_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TextClassificationDataset(test_sentences, test_labels, tokenizer, max_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [35]:
model = BERTClassifier(bert_model_name, num_classes).to(device)

In [36]:
#การกำหนด optimizer และ scheduler ในการฝึกโมเดล BERT
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [39]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/10
Validation Accuracy: 0.4756
              precision    recall  f1-score   support

           0       0.43      0.42      0.43       150
           1       0.49      0.26      0.34       150
           2       0.50      0.75      0.60       150

    accuracy                           0.48       450
   macro avg       0.47      0.48      0.45       450
weighted avg       0.47      0.48      0.45       450

Epoch 2/10
Validation Accuracy: 0.5378
              precision    recall  f1-score   support

           0       0.48      0.64      0.55       150
           1       0.50      0.35      0.41       150
           2       0.64      0.62      0.63       150

    accuracy                           0.54       450
   macro avg       0.54      0.54      0.53       450
weighted avg       0.54      0.54      0.53       450

Epoch 3/10
Validation Accuracy: 0.5422
              precision    recall  f1-score   support

           0       0.50      0.58      0.54       150
           1

In [40]:
torch.save(model.state_dict(), "C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/bert_classifier.pt")

In [46]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        
    sentiment_mapping = {0: "Negative", 1: "Neutrally", 2: "Positive"}
    return sentiment_mapping[preds.item()]

In [52]:
test_text = remove_duplicate_chars("เบื่อสินค้านี้มาก")
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(f"Predicted sentiment: {sentiment}")

Predicted sentiment: Neutrally


In [54]:
tokenized_text = word_tokenize(remove_duplicate_chars(test_text), keep_whitespace=False)
tokenized_text

['เบื่อ', 'สินค้า', 'นี้', 'มาก']

In [58]:
for i in tokenized_text:
    print(i)
    test_text = remove_duplicate_chars(i)
    sentiment = predict_sentiment(test_text, model, tokenizer, device)
    print(f"Predicted sentiment: {sentiment}")
    print("----------------------")

เบื่อ
Predicted sentiment: Neutrally
----------------------
สินค้า
Predicted sentiment: Neutrally
----------------------
นี้
Predicted sentiment: Neutrally
----------------------
มาก
Predicted sentiment: Negative
----------------------
